In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open("/home/davide/Downloads/KEGGbriteCCMEE29.html"), 'html.parser')
text=soup.get_text()
text=text[text.find("ko01001"):]
lines = text.split("\n")
diz={}

for i,line in enumerate(lines):
    if line.startswith("ko"):
        current_key = line[:line.index("(")].strip()
        diz[current_key] = []
    elif line.startswith("WP"):
        proteins = line.split(",") if "," in line else [line]
        diz[current_key].extend(proteins)
from pprint import pprint

import pandas as pd
all_proteins = set(protein for proteins in diz.values() for protein in proteins)
dfs = []
for protein in all_proteins:
    data = {pathway: 1 if protein in proteins else 0 for pathway, proteins in diz.items()}
    df = pd.DataFrame(data, index=[protein])
    dfs.append(df)
result = pd.concat(dfs)
result["KEGG_brite"] = result.apply(lambda row: ", ".join(row[row == 1].index), axis=1)
result.drop(result.columns[:-1], axis=1, inplace=True)


In [6]:
result
#now i want only one column that for each protein says all the pathways it is involved in
result["pathways"] = result.apply(lambda row: ", ".join(row[row == 1].index), axis=1)
result.drop(result.columns[:-1], axis=1, inplace=True)
result

,pathways
WP_250126042.1,ko03000 Transcription factors
WP_250126460.1,ko01008 Polyketide biosynthesis proteins
WP_250125619.1,ko03016 Transfer RNA biogenesis
WP_250122721.1,ko01009 Protein phosphatases and associated pr...
WP_250126199.1,ko01002 Peptidases and inhibitors
...,...
WP_250122320.1,"ko01007 Amino acid related enzymes, ko03016 Tr..."
WP_250125982.1,ko03011 Ribosome
WP_256478939.1,ko02000 Transporters
WP_250124051.1,ko01004 Lipid biosynthesis proteins


In [3]:
fimo_best="CCMEE29best_site.narrowPeak"
genbank_dir = "annotati_Refseq"
import os
import re
from Bio import SeqIO
from collections import Counter
from scipy.stats import hypergeom
import pandas as pd
diz={

    'LAU37' : 'Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff' ,

}
with open(fimo_best,"r") as f:
    content=f.readlines()
names = [line.split()[0] for line in content]
kos = []
kos_background = []
ko_to_genes = {}
i,j=0,0
for file2 in os.listdir(genbank_dir):
    genoma = SeqIO.parse(os.path.join(genbank_dir,file2), "genbank")
    for seq in genoma:
        for record in seq.features:
            if record.type != "CDS":
                continue
            gene=record.qualifiers.get("gene", ["None"])[0]
            pid=record.qualifiers.get("protein_id", ["None"])[0]
            if record.qualifiers["locus_tag"][0] in names:
                i+=1
                #see row of result with the protein id as index
                if pid in result.index:
                    #make a list of columns of the row with value 1
                    ko_terms = result.columns[result.loc[pid] == 1].tolist()
                    for ko_term in ko_terms:
                        kos.append(ko_term)
                        if ko_term in ko_to_genes:
                            ko_to_genes[ko_term].append(gene)
                        else:
                            ko_to_genes[ko_term] = [gene]
            else:
                if pid in result.index:
                    j+=1
                    kos_background.extend(result.columns[result.loc[pid] == 1].tolist())
kos = Counter(kos)
kos_background = Counter(kos_background)
M = i+j
n = i
p_values = {}
enrichments = {}
for ko in kos:
    k = kos[ko]
    N = kos_background[ko]
    p_values[ko] = hypergeom.sf(k-1, M, N, n)
    enrichments[ko] = (k/n) / (N/M)
    enrichments[ko] = "positivo" if enrichments[ko] > 1 else "negativo"
df=pd.DataFrame.from_dict(p_values,orient="index",columns=["p_value"])
df["enrichment"] = pd.Series(enrichments)
for ko in kos:
    df.at[ko, "genes with motif"] = ", ".join(ko_to_genes[ko])
df.sort_values("p_value", inplace=True, ascending=True)

In [4]:
df

,p_value,enrichment,genes with motif
ko03400 DNA repair and recombination proteins,0.112903,positivo,"None, None, radA, lexA, xseA, recA, None, None..."
ko03032 DNA replication proteins,0.496475,positivo,"None, None, topA"
ko03021 Transcription machinery,0.824438,negativo,"None, None"
ko03036 Chromosome and associated proteins,0.892947,negativo,"None, sepF, None, None"
ko01002 Peptidases and inhibitors,0.947833,negativo,"ggt, None, None, lexA, ftsH"
ko03029 Mitochondrial biogenesis,0.949757,negativo,"None, None"
ko03016 Transfer RNA biogenesis,0.960326,negativo,"None, None, None, None, cysS"
ko01007 Amino acid related enzymes,0.960412,negativo,"cysS, bioA"
ko02048 Prokaryotic defense system,0.980930,negativo,"None, None"
ko01011 Peptidoglycan biosynthesis and degradation proteins,0.981009,negativo,None
